# Functions, Types & Dispatch

In this chapter we will review the two key concepts, which make Julia stand out,
namely multiple dispatch and its type system.

## Defining functions and methods

Defining functions follows a rather intuitive syntax. The value obtained by evaluating the last expression of a `function` block will be automatically returned:

In [ ]:
function mymult(x, y)
    x * y
end

For one-line functions one may also use a convenient short-hand:

In [ ]:
mysquare(x) = mymult(x, x)

Both such functions are fully generic in the argument types

In [ ]:
@show mysquare(2)           # Use integer arithmetic
@show mymult(-1, 3. + 2im)  # Use complex arithmetic
@show mysquare(" abc ");    # Use string concatenation

Notice, that for each type combination a separate piece of code will be compiled even though we only *defined* the functionality a single time. This compilation takes place on first use of a particular tuple of types.

Functions are by no means different from other Julia variables

In [ ]:
mymult

and may be passed around to other functions, for example:

In [ ]:
"""The fold function, applying f from the left and accumulating the results"""
function myfold(f, x, y, z)
    f(f(x, y), z)
end
myfold(mymult, "Hello", " Julia ", "World")

Julia makes a distinction between **functions** and **methods**. Roughly speaking **function**s specify *what* is done and **methods** specify *how* this is done.

Methods are concrete implementations in form of a list of Julia expressions to be executed, when the function name is used in the code. Multiple methods may be defined for the same function name. They differ in the number of arguments or in the supported argument types (more on this in a second). When a particular function name is used in the code, Julia looks at the types of the arguments and uses this information to **dispatch** to the best-fitting method. 

For our `myfold` example, one could easily imagine a few more method implementations, for example 

In [ ]:
myfold(f, x) = x
myfold(f, x, y) = f(x, y)

In [ ]:
methods(myfold)

So now `myfold` works transparently with 1, 2 or 3 arguments:

In [ ]:
@show myfold(mymult, 2., 3.)
@show myfold(+, 1)
@show myfold(==, false, false, true)

We can also check which method is actually employed using the `@which` macro:

In [ ]:
@which myfold(*, 1, 2)

*Aside:* Note, that making `myfold` work with a variable number of arguments is possible:

In [ ]:
myfold(f, x, rest...) = myfold(f, f(x, myfold(rest...)))

If you want to know more on this, the `...` is known as *slurping* in the function argument list and as *splatting* in the call towards the end of the line.)

Standard functions (like `+` or `*`) are by no means special and behave exactly the same way as custom functions ... including the ability to define new methods for them:

In [ ]:
import Base: + # we have to import functions to override/extend them
+(x::String, y::String) = x * " " * y

In [ ]:
"Hello" + "World!"

(**Important note:** Since we neither own the `+` function nor the `String` type, this is known as **type piracy** and should in general be avoided!)

Now standard functions relying on `+` just magically work:

In [ ]:
sum(["a", "b", "c", "d", "e"])

##### More details
- https://docs.julialang.org/en/v1/manual/methods/

## Abstract and concrete types

Before we discuss multiple dispatch of functions and dispatch by types, we briefly review Julia's type system. Types in Julia fall into two categories: **Abstract** and **concrete**. Abstract types such as `Integer` or `Number` are supertypes of a bunch of other types, for example:

In [ ]:
Int32 <: Integer   # Read Int32 is-a Integer

In [ ]:
UInt16 <: Integer

In [ ]:
Float32 <: Integer

In [ ]:
Float32 <: Number

In [ ]:
Integer <: Number

In [ ]:
# by transitivity:
@show Int32  <: Number
@show UInt16 <: Number
@show Number <: Number;

### Type properties
We can check type properties in various ways:

In [ ]:
isconcretetype(Int32)

In [ ]:
isabstracttype(Integer)

In [ ]:
1 isa Integer

A fancy way is even to display a type tree ;)

In [ ]:
using AbstractTrees
AbstractTrees.children(x) = subtypes(x)

In [ ]:
print_tree(Number)

In Julia concrete types are always a leaf of the type tree, i.e. they cannot be inherited from each other. For a C++ or Python person (as I was before looking into Julia) this seems restrictive at first, but it takes away a lot of unnecessary complexity from the type hierachy. In Julia the structure of a library or a problem
is in many cases not converted into explict type hierachies,
as it would for OOP languages like Python or Java.
Instead it builds up implicitly from conventions which are associated with abstract or concrete types.

For example, if one implements a concrete type for the abstract type `Number` one is expected to implement a certain set of functions (e.g. `*`, `+`, `-`, `/`, ...). Otherwise not all of the standard library and other linear algebra packages will work. The difference to a hard enforcement of interfaces is, however, that *some things* will still work. This has disadvantages as your code could break in the future, but it is extremely useful for rapidly trying something out.

# Dynamical typing and type deduction

In programming language theory type systems traditionally fall in two categories.
In **dynamically typed** languages the type of
a value or expression is inferred only at runtime,
which usually allows for more flexible code. Examples are Python or MATLAB.
In contrast, so-called **statically-typed** languages (think FORTRAN or C++),
require types to be already known before runtime when the program is compiled.
This allows both to check more thoroughly for errors (which can manifest in mismatched types)
and it usually brings a gain in performance because more things about the memory layout of the program is known
at compile time. As a result aspects such as vectorisation, contiguous alignment of data,
preallocation of memory can be leveraged more easily.

Julia is kind of both. Strictly speaking it is dynamically typed. E.g. the type of variables can change type at any point:

In [ ]:
a = 4
println(typeof(a))
a = "bla"
println(typeof(a))

Note, however, that the type of a *value* cannot change in Julia!

Still, Julia's strong emphasis on types are one of the reasons for its performance.
Unlike in statically typed languages, however, **type deduction in Julia** happens at runtime, right before JIT-compiling a function: The more narrowly the types for a particular piece of code can be deduced, the better the emitted machine code can be optimised. One can influence this using explicit type annotations in function arguments and intermediate expressions. Due to the to the excellent type inference capabilities of Julia, this is in general not needed, however.

This might sound a bit unusal at first, but the result is,
that it takes almost no effort to write generic code as we will see later: Just leave off all the type annotations. Notice, that this only means that the code has no types. At runtime types are still inferred as much as possible, such that aspects like vectorisation, contiguous alignment of data, preallocation of memory *can* be taken into account by the Julia compiler.

Three more facts about Julia types:
- In Julia all types are the same. For example, there is no difference between `Int32` and `String`, even though the first has a direct mapping to low-level instructions in the CPU and the latter has not (contrast this with e.g. C++).
- The `Nothing` type with the single instance `nothing` is the Julia equivalent to `void` in C++ or `None` in Python. It often represents that a function does not return anything or that a variable has not been initialised.
- `Any` is the root of the type tree: Any type in Julia is a subtype of `Any`.

### Exercise
Which of the following type are subtypes of another?
Try to guess first and then verify by using the operator `<:`.

```julia
Float64     AbstractFloat      Integer
Number      AbstractArray      Complex
Real        Any                Nothing
```

##### For more details
https://docs.julialang.org/en/v1/manual/types/

## Multiple dispatch

Let us return back to the `mymult` function:

In [ ]:
mymult(x, y) = x * y

We were able to safely use this functions with a number of type combinations, but some things do not yet work:

In [ ]:
mymult(2, " abc")

Let's say we wanted to concatenate the string `str` $n$ times on multiplication with an integer $n$. In Julia this functionality is already implemented by the exponentiation operator:

In [ ]:
"abc"^4

But for the sake of argument, assume we wanted `mymult("abc", 4)` and `mymult(4, "abc")` to behave the same way. We define two special methods:

In [ ]:
mymult(str::AbstractString, n::Integer) = str^n
mymult(n::Integer, str::AbstractString) = mymult(str, n)

In both of these, the syntax `str::AbstractString` and `n::Integer` means that the respective method is only
considered during dispatch if the argument `str` is of type `AbstractString` or one of its concrete subtypes and similarly `n` is an `Integer` (or subtype). Since Julia always dispatches to the most specific method in case multiple methods match, this is all we need to do:

In [ ]:
mymult(2, " abc")

In [ ]:
@which mymult(2, " abc")

In [ ]:
@which mymult("def ", UInt16(3))

Notice, that the fully generic
```julia
mymult(x, y) = x * y
```
is actually an abbreviation for
```julia
mymult(x::Any, y::Any) = x * y
```

In [ ]:
methods(*)

In [ ]:
@which "Hello"*"World!"

For evaluating such expressions, Julia needs to determine which method of the function `Base.*` to execute.
For this *both* argument types are taken into account and not just the first or the second. This is **multiple dispatch**, namely the fact that for dispatching to a method definition the type of *all* arguments matters.

##### More details
- https://docs.julialang.org/en/v1/manual/methods/

## Standard functions and operators

Plenty of standard functions are already defined in Julia `Base`. This includes:
- All operators `+`, `*`, `-`, `≈` (isapprox)
- Standard functions such as `exp`, `sin`, `cos`, `abs`, ...
- `inv` to compute the multiplicative inverse (e.g. Matrix inverse)
- `min`, `max` and `minmax`

But also a few special cases worth mentioning:
- `cis` for $\exp(i x)$
- `sinpi` and `cospi` for computing $\sin(\pi x)$ and $\cos(\pi x)$ more accurately
- `cbrt` computes the cube root.

##### More details
- https://docs.julialang.org/en/v1/base/math/

### Exercise

Consider the functions

In [ ]:
characterise(a, b::Any)              = "fallback"
characterise(a::Number, b::Number)   = "a and b are both numbers"
characterise(a::Number, b)           = "a is a number"
characterise(a, b::Number)           = "b is a number"
characterise(a::Integer, b::Integer) = "a and b are both integers"

Where `Union{A, B}` means that the type may either be `A` or `B`.

For each of the following calls, try to determine which method will be called (if any) and verify by running the code or using `@which`:
- `characterise(1.5, 2)`
- `characterise(1, "Aachen")`
- `characterise(1, 2)`
- `characterise("Hello", "World!")`
- `characterise(1, true)`

In [ ]:
# For example:
characterise("abc", 1.2)

## Takaways
- Functions (name, what?) have methods (implementation, how?)
- Concrete types are data structures (How is data arranged?)
- Abstract types are informal guarantees (What can I do with it?)
- Multiple dispatch: The most specialised method is selected depending on *all* input types 